In [139]:
import pandas as pd 
import re as re 
import numpy as np
import time 
import multiprocessing
pd.set_option('display.max_colwidth', -1)
pd.set_option('chained_assignment',None)

geneColID = ["chromosome","source","type","start","end","score","strand","phase","gene_symbol","gene_ensID","length","entrezid"]
geneAnnotationDF = pd.read_csv('entrez_id/geneAnnotationsDF_Selected_entrezID.csv', sep=',', comment='#', low_memory=False, header=0, names=geneColID)
chromosomeColID = ['chromosome','source','type','start','end','score','strand','phase']
chromosomesDF = pd.read_csv('chromosomesDF.csv', sep=',', comment='#', low_memory=False, header=0, names=chromosomeColID)

In [140]:
chromosomesDF = chromosomesDF.drop(columns=['score', 'strand', 'phase'])
geneAnnotationDF = geneAnnotationDF.drop(columns=['score', 'phase'])

In [151]:
# Making a list of DataFrames 
u = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','MT','X','Y']
geneDFL = []
for c in u:
    geneDF = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == c]
    geneDFL.append(geneDF)

In [142]:
#I need random chromosome combined with random position (Somewhere between start and end)
def nBindingsites(n):   
    chrIDS = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','MT','X','Y']
    randChr = chromosomesDF.sample(n, replace=True) # Replace allows same row to be sampled again 
    randChr['BindingSite'] = (np.random.uniform(1, randChr.end).round()).astype(int) 
    randChrL = [] 
    for i in chrIDS: 
        DF = randChr.loc[randChr['chromosome'] == i]
        sortDF = DF.iloc[(DF['BindingSite'] - 0).argsort()] 
        randChrL.append(sortDF)
    return randChrL
    #Returns a DataFrame with Random sample number with random BindingSite

In [143]:
#MIGHT NOT NEED ANY OF THIS ACTUALLY COULD BE FASTER 
# Distance from Body
def distanceBody(start, end, site):
    return min(abs(start - site), abs(end - site))

# Distance from TSS
def distanceTSS(start, site):
    return abs(start - site)

#Returns True or False to wheather the binding site is within the window size to Body | Start Site below
def isNearBody(start, end, site, window):
    if abs(start - site) <= window:
        return True
    elif abs(end - site) <= window:
        return True
    else:
        return False 
    
def isNearTSS(start, site, window):
    if abs(start - site) <= window:
        return True
    else: 
        return False
#-------------------------------------------- ## Should there be anymore iterations? Checker function 
def inRange(start, site, window):
    if start > (site + window):
        return False 
    else:
        return True

# For: k = Infinity 

In [144]:
#Given specific chromosome, rdm binding site, window: Will calculate number of nearby gene(bodies) from its site.
def genesInWindowBody(chrNum, site, window):
    count = 0
    geneL = []
    geneDF = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == str(chrNum)]
    
    for index, row in geneDF.iterrows():
        if isNearBody(row['start'], row['end'], site, window):
            count = count + 1
            geneL.append(row['gene_symbol'])
    return(count, geneL)

#Given specific chromosome, rdm binding site, window: Will calculate number of nearby gene(TSS) from the site.
def genesInWindowTSS(chrNum, site, window):
    count = 0
    geneL = []
    geneDF = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == str(chrNum)]
    
    for index, row in geneDF.iterrows():
        if isNearTSS(row['start'], site, window):
            count = count + 1
            geneL.append(row['gene_symbol'])    
    return(count, geneL)

## <font color=#4286f4>Given a window: Find all the genes inside window for the dataframe of chromosomes</font>

In [145]:
# Takes DF and Window size: Returnes Total Gene count and List of Genes Near Body of GENE (K=INFINITY)
def geneCountNsitesBody(df, window):
    count = 0 
    geneL = []
    
    for index, row in df.iterrows():
        c = genesInWindowBody(row['chromosome'], row['BindingSite'], window)
        count = count + c[0]
        geneL.append(c[1])
    return(count, geneL)

# Takes DF and Window size: Returnes Total Gene count and List of Genes Near TSS of GENE (K=INFINITY)
def geneCountNsitesTSS(df, window):
    count = 0 
    geneL = []
    
    for index, row in df.iterrows():
        c = genesInWindowTSS(row['chromosome'], row['BindingSite'], window)
        count = count + c[0]
        geneL.append(c[1])
    return(count, geneL)

# For: k = 1

In [146]:
#Given specific chromosome, rdm binding site, window: Will calculate closest gene(body) from its site. and distance
def closestGeneBody(chrNum, site, window):
    s = time.time()
    geneDF = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == str(chrNum)]
    startG = geneDF.iloc[(geneDF['start'] - site).abs().argsort()].head(1)
    endG = geneDF.iloc[(geneDF['end'] - site).abs().argsort()].head(1)
    bodyG = geneDF[(geneDF['start'] <= site) & (geneDF['end'] >= site)]

    if bodyG.empty == False: 
        return(bodyG['entrezid'].iloc[0])
    
    startDist = distanceBody(startG['start'].iloc[0], startG['end'].iloc[0], site)
    endDist = distanceBody(endG['start'].iloc[0], endG['end'].iloc[0], site)
    if startDist < endDist:
        if isNearBody(startG['start'].iloc[0], startG['end'].iloc[0], site, window):
            return(startG['entrezid'].iloc[0])
        else:
            return('No Result')
    else:
        if isNearBody(endG['start'].iloc[0], endG['end'].iloc[0], site, window):
            return(endG['entrezid'].iloc[0])
        else:
            return('No Result')
        
def closestGeneBody2(site, count, window):
    geneDF = geneDFL[count]
    startG = geneDF.iloc[(geneDF['start'] - site).abs().argsort()].head(1)
    endG = geneDF.iloc[(geneDF['end'] - site).abs().argsort()].head(1)
    bodyG = geneDF[(geneDF['start'] <= site) & (geneDF['end'] >= site)]
    
    if bodyG.empty == False: 
        return(bodyG['entrezid'].iloc[0])
    
    startDist = distanceBody(startG['start'].iloc[0], startG['end'].iloc[0], site)
    endDist = distanceBody(endG['start'].iloc[0], endG['end'].iloc[0], site)
    if startDist < endDist:
        if isNearBody(startG['start'].iloc[0], startG['end'].iloc[0], site, window):
            return(startG['entrezid'].iloc[0])
        else:
            return('No Result')
    else:
        if isNearBody(endG['start'].iloc[0], endG['end'].iloc[0], site, window):
            return(endG['entrezid'].iloc[0])
        else:
            return('No Result')
        

#Given specific chromosome, rdm binding site, window: Will calculate closest gene(TSS) from its site. and Distance
def closestGeneTSS(chrNum, site, window):
    geneDF = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == str(chrNum)]
    startG = geneDF.iloc[(geneDF['start'] - site).abs().argsort()].head(1)
    if startG.empty: 
        return('No Result')
    elif isNearTSS(startG['start'].iloc[0], site, window):
        return(startG['entrezid'].iloc[0])
    else:
        return('No Result')

In [147]:
# Takes DF and Window size: Returnes Total Gene count and List of CLOSEST Gene Near its BODY for every site(K = 1)
def closestGenesNsitesBody(df, window):
    geneL = []   
    for row in zip(df['chromosome'].astype(str), df['BindingSite'].values):
        x = closestGeneBody(row[0], row[1], window)
        if x == 'No Result':
            pass
        else:
            geneL.append(x)
    return(geneL)
    
def closestGenesNsitesBody2(DFL_BS, window):
    geneList = []
    for v in range(len(DFL_BS)):
        d = DFL_BS[v]
        if d.empty == False:
            for row in zip(d['BindingSite'].values):
                x = closestGeneBody2(row[0], v, window)
                geneList.append(x)         
    return geneList
     
# Takes DF and Window size: Returnes Total Gene count and List of CLOSEST Gene Near its TSS for every site (K = 1)
def closestGenesNsitesTSS(df, window):
    count = 0 
    geneL = []
    
    for index, row in df.iterrows():
        x = closestGeneTSS(row['chromosome'], row['BindingSite'], window)
        geneL.append(x[0])
        count = count + 1
    return(count, geneL)

In [148]:
def closestGenesNsitesBodyFASTER(df, window):
    num_processes = multiprocessing.cpu_count()
    chunk_size = int(df.shape[0]/num_processes)
#     chunks = [df.ix[df.index[i:i + chunk_size]] for i in range(0, df.shape[0], chunk_size)]
    chunks = [df.iloc[i:i + chunk_size,:] for i in range(0, df.shape[0], chunk_size)]
    return chunks
#     geneL = []   
#     for row in zip(df['chromosome'].astype(str), df['BindingSite'].values):
#         x = closestGeneBody(row[0], row[1], window)
#         if x == 'No Result':
#             pass
#         else:
#             geneL.append(x)
#     return(geneL)

In [153]:
# Adds a window range by adding COl lowB and upperB 
def addWindow(window):
    geneWindowDF = geneDFL.copy()
    for d in geneWindowDF:
        d['lowB'] = d.start - window
        d['upperB'] = d.end + window
    return(geneWindowDF)

In [405]:
def isinRange(x, lowB, upperB):
    if (x >= lowB) & (x <= upperB):
        return True 
    else:
        return False

def closestGeneNSitesBody(DFL_BS, window):
    groupedGenes = addWindow(window)
    outputGeneL = []
    pos = 0 # position in the DF we are looking at 
    
    for sitesDF in DFL_BS: # DFL_BS is dataframe of chomosomes with N_BS ordered by the chromosome number  
        cutoff = 0 # Don't need to search above this index so cut it off
        
        for site in zip(sitesDF['BindingSite'].values):
            endCut = len(groupedGenes[pos])
            df = groupedGenes[pos].iloc[cutoff:endCut]  #endCut is the length of groupedGenes[pos]
            gene = df.loc[((df.lowB - site[0]) * (df.upperB - site[0])) < 0]#.head(1)
            if gene.empty == True:
                outputGeneL.append('No Result')
            else:
                outputGeneL.append(gene.iat[0,9])
        pos = pos + 1
    return outputGeneL

In [411]:
g = nBindingsites(10000)

In [414]:
s = time.time()
out = closestGeneNSitesBody(g, 90)
time.time() - s

11.959319114685059

In [ ]:
def closestGeneNSitesBodyCUTOFF(DFL_BS, window):
    groupedGenes = addWindow(window)
    outputGeneL = []
    pos = 0 # position in the DF we are looking at 
    
    for sitesDF in DFL_BS: # DFL_BS is dataframe of chomosomes with N_BS ordered by the chromosome number  
        cutoff = 0 # Don't need to search above this index so cut it off
        
        for site in zip(sitesDF['BindingSite'].values):
            endCut = len(groupedGenes[pos])
            df = groupedGenes[pos].iloc[cutoff:endCut]  #endCut is the length of groupedGenes[pos]
            gene = df.loc[((df.lowB - site[0]) * (df.upperB - site[0])) < 0]#.head(1)
            if gene.empty == True:
                outputGeneL.append('No Result')
            else:
                outputGeneL.append(gene.iat[0,9])
        pos = pos + 1
    return outputGeneL